## StateFarm Kaggle Challenge

- For this Kaggle Challenge, I will be performing a rigorous analysis of the dataset provided by StateFarm to predict whether or not the image of a driver is in a distracted or non distracted phase.


- StateFarm has provided labelled training data in the form of images of drivers that have been classified in one of 10 different states.


- The states of the drivers are :
    - c0 : Safe Driving (2489 images)
    - c1 : texting - right (2267 images)
    - c2 : talking on the phone - right (2317 images)
    - c3 : texting - left (2346 images)
    - c4 : talking on the phone - left (2326 images)
    - c5 : operating the radio (2312 images)
    - c6 : drinking (2325 images)
    - c7 : reaching behind (2002 images)
    - c8 : hair and makeup (1911 images)
    - c9 : talking to a passenger (2129 images)


- The testing data provided is totally unlabelled as expected.
    

- My goal for this notebook is to demonstrate an intuitive understanding of going about solving a computer vision problem.


- I will be solving this problem by building on top of the Vgg16 model and I will be employing various proven methods that improve accuracy. I will not be going into the mathematical details of approaches, but rather something that can be thought through intuitively such that the process adds up.

## Steps :

- Creating validation + sample sets
- Rearranging image files into respective directories
- Finetuning & Training model
- Generating Predictions
- Validating Predictions
- Submitting to Kaggle

In [3]:
%mkdir valid
%mkdir results
%mkdir -p sample/train
%mkdir -p sample/valid
%mkdir -p sample/results
%mkdir -p test/unknown

/home/ubuntu/nbs/data/statefarm


In [5]:
for d in glob('c?'):
    os.mkdir('../sample/train/'+d)
    os.mkdir('../sample/valid/'+d)
    os.mkdir('../valid/'+d)

In [6]:
# Separated 1950 out of 22424 images from the training set to the validation set.
g = glob('c?/*.jpg')
shuf = np.random.permutation(g)
for i in range(2000):
    os.rename(shuf[i], DATA_DIRECTORY+'/valid/'+shuf[i])

In [7]:
from shutil import copyfile

In [8]:
# Creating sample data from training & validation data to run as a test for quick iteration
g = glob('c?/*.jpg')
shuf = np.random.permutation(g)
for i in range(1500): copyfile(shuf[i], '../sample/train/'+shuf[i])

In [9]:
%cd ../valid

/home/ubuntu/nbs/data/statefarm/valid


In [10]:
g = glob('c?/*.jpg')
shuf = np.random.permutation(g)
for i in range(1000): copyfile(shuf[i], '../sample/valid/' + shuf[i])

In [28]:
%ls

driver_imgs_list.csv.zip  results/  sample_submission.csv.zip  train/
imgs.zip                  sample/   test/                      valid/


## Creating Batches

In [1]:
from theano.sandbox import cuda

from __future__ import print_function, division
path = "data/statefarm/"
import utils; reload(utils)
from utils import *

batch_size = 64

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [2]:
vgg = Vgg16()
model = vgg.model
conv_layers, _ = split_at(model, Convolution2D)

conv_model = Sequential(conv_layers)

In [4]:
conv_layers

In [5]:
print("Retrieving batches...")
batches = get_batches(path+'train', batch_size=batch_size)
print("Retrieving val_batches...")
val_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=False)
print("retrieving test_batches...")
test_batches = get_batches(path+'test', batch_size=batch_size*2, shuffle=False)

Retrieving batches...
Found 20424 images belonging to 10 classes.
Retrieving val_batches...
Found 2000 images belonging to 10 classes.
retrieving test_batches...
Found 79726 images belonging to 1 classes.


In [6]:
print("Retrieving classes, labels and filenames...")
(val_classes, trn_classes, val_labels, trn_labels, val_filenames, filenames, test_filenames) = get_classes(path)

Retrieving classes, labels and filenames...
Found 20424 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


In [ ]:
print("Precomputing Convolution Output from the Vgg model...")
conv_feat = conv_model.predict_generator(batches, batches.nb_sample)
# conv_val_feat = conv_model.predict_generator(val_batches, val_batches.nb_sample)
conv_test_feat = conv_model.predict_generator(test_batches, test_batches.nb_sample)

Precomputing Convolution Output from the Vgg model...


In [ ]:
print("Saving pre-computed convolutional data")
save_array(path+'results/conv_feat.dat', conv_feat)
save_array(path+'results/conv_val_feat.dat', conv_val_feat)
save_array(path+'results/conv_test_feat.dat', conv_test_feat)

In [ ]:
print("Loading pre-computed convolutional data")
conv_feat = load_array(path+'results/conv_feat.dat')
conv_val_feat = load_array(path+'results/conv_val_feat.dat')

In [ ]:
# Since the pre-computation of the convolutional network is done, the next step is to create a network that takes the
# conv output as its input and predicts the 10 driver classes...

# A dropout value
p=0.8

def get_bn_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dropout(p/2),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(p/2),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(10)
    ]

In [ ]:
# Precomp_model takes in the
def precomp_model(conv_feat, trn_labels, conv_val_feat, val_labels):
    print("Getting the final classification layers")
    bn_model = Sequential(gen_bn_layers(p))
    bn_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

    print("Fitting the bn_model with a learning rate of 0.001")
    bn_model.fit(da_conv_feat, da_trn_labels, batch_size= batch_size, nb_epoch = 1, validation_data=(conv_val_feat, val_labels))

    print("Fitting the bn_model with a learning rate of 0.01")
    bn_model.optimizer.lr =0.01
    bn_model.fit(da_conv_feat, da_trn_labels, batch_size= batch_size, nb_epoch = 2, validation_data=(conv_val_feat, val_labels))

    print("Model weights are saved in the models directory...")
    bn_model.save_weights(path+'models/precomp_model.h5')

    return bn_model

In [ ]:
print("Compiling precomp_model...")
precomp_model(conv_feat, trn_labels, conv_val_feat, val_labels)

In [ ]:
# Preparing submission...
print("Preparing a submission file...")
def do_clip(arr, mx):
    return np.clip(arr, (1-mx)/9, mx)

# precomputed convolutional test output
conv_test_feat = load_array(path+'results/conv_test_feat.dat')

# returns a numpy array of predictions
predictions = bn_model.predict(conv_test_feat, batch_size=batch_size*2)

subm = do_clip(predictions, 0.93)

submission_name = path+'results/toSubmit1.csv'

# classes
classes = sorted(batches.class_indices, key=batches.class_indices.get)

# CSV submission reay file
submission = pd.DataFrame(subm, columns=classes)
submission.insert(0, 'img', [a[4:] for a in test_filenames])

submission.to_csv(submission_name, index=False, encoding='utf-8')
print("Donezo...")